In [2]:
import pandas as pd
from sqlalchemy import create_engine,func

In [3]:
# Example: SQLite (change this to your DB)
engine = create_engine("sqlite:///mydatabase.db")  

In [4]:
# 1. Deal stage='Won' consider krna chahiye

#### How is each sales team performing compared to the rest?

In [5]:
query ="""WITH Manager_Data as (
SELECT manager, ROUND(SUM(close_value),0) as Revenue,COUNT(opportunity_id) as Total_Leads,
100*COUNT(CASE WHEN deal_stage = 'Won' THEN 1 END)/COUNT(opportunity_id) AS Conversion_Rate,
ROUND(SUM(CASE WHEN deal_stage = 'Won' THEN close_value END)/SUM(CASE WHEN deal_stage = 'Won' THEN 1 END),2) 
as Average_Deal_Size
FROM sales_teams t1 LEFT JOIN sales_pipeline t2 ON t1.sales_agent = t2.sales_agent
group by t1.manager)

SELECT manager,Revenue,RANK() OVER (ORDER BY Revenue DESC) as Revenue_Rank,Total_Leads,
RANK() OVER (ORDER BY Total_Leads DESC) as Leads_Rank,Conversion_Rate,
RANK() OVER (ORDER BY Conversion_Rate DESC) as Conversion_Rank,Average_Deal_Size,
RANK() OVER (ORDER BY Average_Deal_Size DESC) as Deal_Size_Rank
FROM Manager_Data
"""

with engine.connect() as conn:
    df = pd.read_sql(query, conn)

print(df)


            manager    Revenue  Revenue_Rank  Total_Leads  Leads_Rank  \
0     Melvin Marxen  2251930.0             1         1929           1   
1     Summer Sewald  1964750.0             2         1701           2   
2     Rocco Neubert  1960545.0             3         1327           4   
3      Celia Rouche  1603897.0             4         1296           5   
4        Cara Losch  1130049.0             5          964           6   
5  Dustin Brinkmann  1094363.0             6         1583           3   

   Conversion_Rate  Conversion_Rank  Average_Deal_Size  Deal_Size_Rank  
0               45                6            1588.10               3  
1               48                3            1526.61               4  
2               52                1            1761.50               1  
3               47                4            1667.25               2  
4               49                2            1516.84               5  
5               47                4             92

#### Are any sales agents lagging behind?

In [6]:
query ="""WITH Agent_Data as (
SELECT sales_agent, ROUND(SUM(close_value),0) as Revenue,COUNT(opportunity_id) as Total_Leads,
100*COUNT(CASE WHEN deal_stage = 'Won' THEN 1 END)/COUNT(opportunity_id) AS Conversion_Rate,
ROUND(SUM(CASE WHEN deal_stage = 'Won' THEN close_value END)/SUM(CASE WHEN deal_stage = 'Won' THEN 1 END),2) 
as Average_Deal_Size
FROM sales_pipeline 
group by sales_agent)

SELECT sales_agent,Revenue,RANK() OVER (ORDER BY Revenue DESC) as Revenue_Rank,Total_Leads,
RANK() OVER (ORDER BY Total_Leads DESC) as Leads_Rank,Conversion_Rate,
RANK() OVER (ORDER BY Conversion_Rate DESC) as Conversion_Rank,Average_Deal_Size,
RANK() OVER (ORDER BY Average_Deal_Size DESC) as Deal_Size_Rank
FROM Agent_Data
"""

with engine.connect() as conn:
    df = pd.read_sql(query, conn)

print(df)


           sales_agent    Revenue  Revenue_Rank  Total_Leads  Leads_Rank  \
0      Darcel Schlecht  1153214.0             1          747           1   
1       Vicki Laflamme   478396.0             2          451           2   
2      Kary Hendrixson   454298.0             3          438           4   
3         Cassey Cress   450489.0             4          346           8   
4        Donn Cantrell   445860.0             5          275          16   
5         Reed Clapper   438336.0             6          237          22   
6            Zane Levy   430068.0             7          349           7   
7        Corliss Cosme   421036.0             8          310          12   
8       James Ascencio   413533.0             9          267          17   
9      Daniell Hammack   364229.0            10          259          20   
10     Maureen Marcano   350395.0            11          285          14   
11    Gladys Colclough   345674.0            12          317          10   
12      Mark

#### Do any products have better win rates?

In [33]:
query ="""WITH Product_Data as (SELECT product,COUNT(CASE WHEN deal_stage = 'Won' THEN 1 END) as Deals_Won,
COUNT(CASE WHEN deal_stage != 'Won' THEN 1 END) as Deals_Lost FROM sales_pipeline group by product)

SELECT product,ROUND(100.0*Deals_Won/(Deals_Won+Deals_Lost),2) as Win Rate FROM Product_Data 
 """

with engine.connect() as conn:
    df = pd.read_sql(query, conn)

print(df)


          product  Win Rate
0         GTK 500     37.50
1       GTX Basic     49.04
2  GTX Plus Basic     47.22
3    GTX Plus Pro     49.48
4          GTXPro     49.26
5     MG Advanced     46.32
6      MG Special     48.03


#### Are there any quarter-over-quarter trends?

In [29]:
query ="""WITH Quarter_Data as (SELECT strftime('%Y', close_date) AS year,
(((CAST(strftime('%m', close_date) AS INTEGER) - 1) / 3) + 1) AS quarter,
SUM(close_value) as Revenue
FROM sales_pipeline WHERE deal_stage = 'Won' GROUP BY year,quarter),
Quarter_Lag_Data as (SELECT quarter,Revenue,IFNULL(LAG(Revenue) OVER (ORDER BY quarter),0) as Previous_Quarter_Revenue 
FROM Quarter_Data)
SELECT quarter,ROUND(100*(Revenue-Previous_Quarter_Revenue)/Previous_Quarter_Revenue,2) as QoQ_Change 
FROM Quarter_Lag_Data WHERE Previous_Quarter_Revenue!=0"""

with engine.connect() as conn:
    df = pd.read_sql(query, conn)

print(df)

   quarter  QoQ_Change
0        2      171.98
1        3       -3.37
2        4       -6.03
